<a href="https://colab.research.google.com/github/ArghyaPal/medical_image/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generator architecture

In [0]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        # input is Z, going into a convolution
        self.dconv1 =  nn.ConvTranspose2d( nz, ngf * 8, 3, 1, 0, bias=False)
        self.batch1 = nn.BatchNorm2d(ngf * 8)
        self.relu =   nn.ReLU(True)
        # state size. (ngf*8) x 4 x 4
        self.dconv2 =  nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False)
        self.batch2 = nn.BatchNorm2d(ngf * 4)
        # state size. (ngf*4) x 8 x 8
        self.dconv3 =  nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False)
        self.batch3 = nn.BatchNorm2d(ngf * 2)
        # state size. (ngf*2) x 16 x 16
        self.dconv4 =  nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False)
        self.batch4 =  nn.BatchNorm2d(ngf)
        self.dconv5 =  nn.ConvTranspose2d(ngf, ngf, 4, 2, 1, bias=False)
        # state size. (ngf) x 32 x 32
        self.dconv6 = nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False)
        self.tan =   nn.Tanh()
        # state size. (nc) x 64 x 64

    def forward(self, input):
        x = self.dconv1(input)
        #print("I am Generator")
        #print(x.size)
        x = self.batch1(x)
        x = self.relu(x)
        x = self.dconv2(x)
        #print(x.shape)
        x = self.batch2(x)
        x = self.relu(x)
        x = self.dconv3(x)
        #print(x.shape)
        x = self.batch3(x)
        x = self.relu(x)
        x = self.dconv4(x)
        #print(x.shape)
        x = self.batch4(x)
        x = self.relu(x)
        x = self.dconv5(x)
        #print(x.shape)
        x = self.dconv6(x)
        #print(x.shape)
        x = self.tan(x)
        #print("EOF G")
        return x

## Discriminator architecture

In [0]:
# Discriminator Code

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        # input is (nc) x 64 x 64
        self.conv1 =  nn.Conv2d(nc, ndf, 4, 2, 1, bias=False)
        self.lrelu1 = nn.LeakyReLU(0.2, inplace=True)
        # state size. (ndf) x 32 x 32
        self.conv2 =  nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False)
        self.batch1 = nn.BatchNorm2d(ndf * 2)
           #nn.LeakyReLU(0.2, inplace=True),
        # state size. (ndf*2) x 16 x 16
        self.conv3 =  nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False)
        self.batch2 = nn.BatchNorm2d(ndf * 4)
            #nn.LeakyReLU(0.2, inplace=True),
        # state size. (ndf*4) x 8 x 8
        self.conv4 =  nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False)
        self.batch3 = nn.BatchNorm2d(ndf * 8)
            #nn.LeakyReLU(0.2, inplace=True),
        # state size. (ndf*8) x 4 x 4
        self.conv5 =  nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False)
        self.linear = nn.Linear(900,100)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        x = self.conv1(input)
        #print("I am Discriminator")
        #print(x.shape)
        x = self.lrelu1(x)
        x = self.conv2(x)
        #print(x.shape)
        x = self.batch1(x)
        x = self.lrelu1(x)
        x = self.conv3(x)
        #print(x.shape)
        x = self.batch2(x)
        x = self.lrelu1(x)
        x = self.conv4(x)
        #print(x.shape)
        x = self.batch3(x)
        x = self.lrelu1(x)
        x = self.conv5(x)
        #print(x.shape)
        x = x.view(100*1*3*3)
        x = self.linear(x)
        x = self.sigmoid(x)
        #print("EOF D")
        return x